### Setting

In [55]:
# import libraries
import ipfshttpclient
import json

from datetime import datetime
from pathlib import Path
from web3 import Web3

In [68]:
# IPFSノードに接続
client = ipfshttpclient.connect('/ip4/127.0.0.1/tcp/5001')

### 1. IPFSに画像ファイルをアップロードする

In [70]:
def upload_to_ipfs(path):
    """ Upload image to IPFS server

    Args:
        path: file path

    Returns:
        CID representing the location of the file returned by the IPFS server    
    """
    res = client.add(path)
    return res['Hash']

In [123]:
image_name = 'sample003.jpg'

hash = upload_to_ipfs('../img/' + image_name)
print(hash)

QmUtG2VdpALNLCnThcHe2ukguEgPLw6b8jBucWG8ZpbfCa


### 2. メタデータを作成しIPFSにアップロードする

In [124]:
def create_metadata(attributes, hash):
    """ Create metadata for registration in a transaction

    Args:
        attributes: Information to be described in metadata
        hash: CID of image data

    Returns:
        Dictionary type metadata 
    """

    # 日付文字列をUNIX時間（エポック秒）に変換
    date_object = datetime.strptime(attributes['date'], "%Y-%m-%d")
    unix_time = int(date_object.timestamp())

    metadata = {
        "name": attributes['name'],
        "description": attributes['description'],
        "image": f"https://ipfs.io/ipfs/{hash}",
        "attributes": [
            {
                "trait_type": "institution",
                "value": attributes['institution']
            },
            {
                "display_type": "date", 
                "trait_type": "date", 
                "value": unix_time
            },
            {
                "trait_type": "score",
                "value": attributes['score']
            }
        ]
    }

    return metadata

In [125]:
# 資格情報を定義（フロント側から受け取る想定）
attributes = {
    'name': "2級ファイナンシャル・プランニング技能士",
    'description': "web3 hackathon",
    "institution": "日本FP協会",
    'date': "2024-02-01",
    'score': 1
}
metadata = create_metadata(attributes, hash)
print(metadata)

# JSONファイルに書き出す
with open("../metadata/metadata.json", "w") as json_file:
    json.dump(metadata, json_file)

{'name': '2級ファイナンシャル・プランニング技能士', 'description': 'web3 hackathon', 'image': 'https://ipfs.io/ipfs/QmUtG2VdpALNLCnThcHe2ukguEgPLw6b8jBucWG8ZpbfCa', 'attributes': [{'trait_type': 'institution', 'value': '日本FP協会'}, {'display_type': 'date', 'trait_type': 'date', 'value': 1706713200}, {'trait_type': 'score', 'value': 1}]}


In [126]:
# メタデータをIPFSサーバーにアップロード
hash_metadata = upload_to_ipfs('../metadata/metadata.json')
print(hash_metadata)

QmY4QC9dkjhz5NWUSEN2kK1S6KscnCVF9Pnn1nW7wc1uHo


### 3. トランザクションの発行
Remixで実施する

In [127]:
# safeMint関数に入力するURI
print(f"https://ipfs.io/ipfs/{hash_metadata}")

https://ipfs.io/ipfs/QmY4QC9dkjhz5NWUSEN2kK1S6KscnCVF9Pnn1nW7wc1uHo


### 4. スマートコントラクトの実行

In [128]:
def get_contract(contract_address, provider_url):
    """Create instance for smart contracts"""

    # ABI（関数のインターフェース）を指定
    contract_abi = [{
        "name": "tokenURI",
        "inputs": [{"name": "tokenId", "type": "uint256"}], 
        "outputs": [{"name": "", "type": "string"}],
        "constant": True, 
        "payable": False,
        "stateMutability": "view",
        "type": "function"
    }]
    web3 = Web3(Web3.HTTPProvider(provider_url))
    contract = web3.eth.contract(address=contract_address, abi=contract_abi)
    return contract

In [129]:
# パラメータ設定
contract_address = '0xe34A19c0a479E9C38F7975C0802e296fCF3652a1'
provider_url = "https://eth-sepolia.g.alchemy.com/v2/WcNDGCbds2ZYFi1SZ-RgpHRyxO6KaugU"

# スマートコントラクトのインスタンスを作成
contract = get_contract(contract_address, provider_url)

# tokenURI関数を呼び出しURIを取得
token_id = 4  # 取得したいトークンのID
uri = contract.functions.tokenURI(token_id).call()
print('Token URI:', uri)

Token URI: https://ipfs.io/ipfs/QmY4QC9dkjhz5NWUSEN2kK1S6KscnCVF9Pnn1nW7wc1uHo


### 5. メタデータを取得する

In [130]:
# URI内のCIDをもとにメタデータを取得
data = client.get_json(uri.split('/')[-1])

data

{'name': '2級ファイナンシャル・プランニング技能士',
 'description': 'web3 hackathon',
 'image': 'https://ipfs.io/ipfs/QmUtG2VdpALNLCnThcHe2ukguEgPLw6b8jBucWG8ZpbfCa',
 'attributes': [{'trait_type': 'institution', 'value': '日本FP協会'},
  {'display_type': 'date', 'trait_type': 'date', 'value': 1706713200},
  {'trait_type': 'score', 'value': 1}]}